In [13]:
!pip install openai pydantic
import json, textwrap

In [14]:
from google.colab import userdata, drive
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [15]:
raw_text = """
Of Man’s first disobedience, and the fruit
Of that forbidden tree whose mortal taste
Brought death into the World, and all our woe,
With loss of Eden, till one greater Man
Restore us, and regain the blissful seat,
Sing Heavenly Muse, that, on the secret top
Of Oreb or of Sinai, didst inspire
That shepherd who first taught the chosen seed,
In the beginning how the heavens and earth
Rose out of Chaos: or if Sion Hill
Delight thee more, and Siloa’s brook that flow’d
Fast by the oracle of God; I thence
Invoke thy aid to my advent’rous song,
That with no middle flight intends to soar
Above th’ Aonian mount, while it pursues
Things unattempted yet in prose or rhyme.
"""

class PLExcerpt(BaseModel):
    book: int
    lines: str
    characters: list[str]
    themes: list[str]
    summary: str

    model_config = {"extra": "forbid"}

schema = PLExcerpt.model_json_schema()

In [19]:
response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {
            "role": "system",
            "content": "Extract structured data about the passage. Make sure the summary is concise and technical, fitting for an academic well-versed in the text. Remove any filler or aggrandizement."
                       "Return JSON only—no extra keys or text."
        },
        {
            "role": "user",
            "content": raw_text
        },
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "paradise_lost_excerpt",
            "strict": True,
            "schema": schema,
        }
    },
)

print("Parsed object:\n")

parsed = json.loads(response.output_text)
parsed["summary"] = textwrap.fill(parsed["summary"], width=80)
print(json.dumps(parsed, indent=2))

Parsed object:

{
  "book": 1,
  "lines": "1-16",
  "characters": [
    "Heavenly Muse"
  ],
  "themes": [
    "Fall of Man",
    "Disobedience",
    "Redemption",
    "Divine Inspiration"
  ],
  "summary": "The passage serves as the invocation to the muse, detailing the themes of\nhumanity's original sin and the promise of redemption through a greater figure.\nThe narrator seeks divine inspiration to recount events of divine magnitude,\nsurpassing prior literary efforts."
}
